In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from functools import reduce
import re

In [ ]:
def get_tables(path):
    """"
    :param: path of the data base
    :return: data frame with all the data, united by 'uuid'.
    """
    engine = create_engine(f'sqlite:///{path}')
    df_personal_info = pd.read_sql("SELECT * FROM personal_info", engine)
    df_country_info = pd.read_sql("SELECT * FROM country_info", engine)
    df_career_info = pd.read_sql("SELECT * FROM career_info", engine)
    df_poll_info = pd.read_sql("SELECT * FROM poll_info", engine)

    # getting the complete data frame
    dfs_list = [df_personal_info, df_country_info, df_career_info, df_poll_info]
    df_info = reduce(lambda left, right: pd.merge(left, right, on='uuid'), dfs_list)

    # exporting the data frame to processed data folder.
    #df_info.to_csv(f'../../data/processed/info_dataframe.csv', index=True)

    return df_info

In [ ]:
get_tables('../data/raw/raw_data_project_m1.db')

In [ ]:
path = '../data/raw/raw_data_project_m1.db'
engine = create_engine(f'sqlite:///{path}')
df_personal_info = pd.read_sql("SELECT * FROM personal_info", engine)
df_country_info = pd.read_sql("SELECT * FROM country_info", engine)
df_career_info = pd.read_sql("SELECT * FROM career_info", engine)
df_poll_info = pd.read_sql("SELECT * FROM poll_info", engine)

# getting the complete data frame
dfs_list = [df_personal_info, df_country_info, df_career_info, df_poll_info]
df_complete = reduce(lambda left, right: pd.merge(left, right, on='uuid'), dfs_list)

# exporting the data frame to processed data folder.
#df_complete.to_csv(f'./data/processed/complete_dataframe.csv', index=True)
df_complete

In [ ]:
# cleaning personal_info gender column.
df_complete['gender'] = df_complete['gender'].astype("string").str.capitalize()
df_complete['gender'] = df_complete['gender'].replace('Fem', 'Female')

In [ ]:
# cleaning personal_info age_group column.
df_complete['age_group'] = df_complete['age_group'].replace('juvenile', '14_25')

In [ ]:
# cleaning personal_info age column.
df_complete['age'] = df_complete['age'].astype("string")
df_complete['age'] = df_complete['age'].str.replace(' years old', '')

for x in range(1980, 2050):
    df_complete['age'] = df_complete['age'].str.replace(f'{x}', f'{2016 - x}')

df_complete['age'] = df_complete['age'].astype('int64')

In [ ]:
df_complete

In [ ]:
#Hacemos filtro para ver mejor los elemntos correspondientes a 'juvenile'. Max y min pertenecen al bin 14-25, 
#porque 2016-1991 = 25 y 2016-2002=14, por lo que entendemos que todos pertecen a la msima categoría.
#juvenile_filter = df_personal_info['age_group'].str.contains("juvenile")
#print(df_personal_info[juvenile_filter].max())
#print(df_personal_info[juvenile_filter].min())

In [ ]:
# cleaning country_info rural column.
df_complete['rural'] = df_complete['rural'].str.lower()

In [ ]:
# cleaning career_info dem_education_level column.
df_complete['dem_education_level'] = df_complete['dem_education_level'].replace('no', 'no education')
df_complete['dem_education_level'] = df_complete['dem_education_level'].fillna('no education')

In [ ]:
# cleaning career_info dem_education_level column.
df_complete['dem_education_level'] = df_complete['dem_education_level'].replace('no', 'no education')
df_complete['dem_education_level'] = df_complete['dem_education_level'].fillna('no education')

In [ ]:
# cleaning def_has_chidren
df_complete['dem_has_children'] = df_complete['dem_has_children'].str.lower()
df_complete.head()

In [ ]:
#connecting API
import json
import requests

In [ ]:
response = requests.get(f'http://api.dataatwork.org/v1/jobs') #ESTO NO LO HE PUESTO EN PIPELINE
json_data = response.json()
#problema: solo me salen 20

In [ ]:
#response = requests.get('http://api.dataatwork.org/v1/jobs/uuid')
#results = response.json()
#print(len(results))
#solo saca 21 pero necesito más. lo que vamos a hacer es que nos saque los datos 
# para cada uuid, para eso necesitamos saber cuántos uuids hay.

In [ ]:
job_code_unique = df_complete['normalized_job_code'].unique()
#len(job_code_unique)

In [ ]:
list_uuid_key = []
list_title_value = []

for code in job_code_unique:
    response = requests.get(f'http://api.dataatwork.org/v1/jobs/{code}').json()
    if list(response.keys())[0] == 'error':
        pass
    else:
        list_uuid_key.append(response['uuid'])
        list_title_value.append(response['title'])

In [ ]:
dict_uuid_jobs_title = dict(zip(list_uuid_key , list_title_value))

In [ ]:
df_complete['Job Title'] = df_complete['normalized_job_code']

for uuid, title in dict_uuid_jobs_title.items():
    df_complete.loc[df_complete['normalized_job_code'] == uuid , 'Job Title'] = title

In [ ]:
df_complete

Web scraping

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
url = 'https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes'
html = requests.get(url).content
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
table = soup.find_all('div' , {'class': 'mw-content-ltr'})[0]
table_countries = table.find_all('table')
all_info_countries = [info.text for info in table_countries]

In [ ]:
# cleaning the info countries.
countries_without_spaces = [re.sub(r'\s' , '' , x) for x in all_info_countries]
countries_without_symbols = [re.sub(r'\*' , '' , x) for x in countries_without_spaces]
countries_without_squarebrackets = [re.sub(r'\[\d\]' , '' , x) for x in countries_without_symbols]
countries_clean = ''.join(countries_without_squarebrackets) #unimos todo en una string para poder hacer split

In [ ]:
country_value = re.split(r'\(\w{0,7}\)' , countries_clean)
country_key = re.findall(r'\(\w{0,7}\)' , countries_clean) #extract the codes
country_key = [re.sub(r'\(|\)' , '' , x) for x in country_key] #without parenthesis.
dic_countries = dict(zip(country_key, country_value))

In [ ]:
df_complete['Country'] = df_complete['country_code']

for code, name in dic_countries.items():
    df_complete.loc[df_complete['country_code'] == code , 'Country'] = name

In [ ]:
df_complete.head()

In [ ]:
# renaming columns
df_complete.rename(columns={'age': 'Age',
                           'gender': 'Gender',
                           'dem_has_children': 'Childrens',
                           'age_group': 'Age Group',
                           'country_code': 'Country Code',
                           'rural': 'Area',
                           'dem_education_level': 'Education Level',
                           'dem_full_time_job': 'Full Time Job',
                           'normalized_job_code': 'Job Code',
                           'question_bbi_2016wave4_basicincome_awareness': 'Q1: awareness',
                           'question_bbi_2016wave4_basicincome_vote': 'Q2: vote',
                           'question_bbi_2016wave4_basicincome_effect': 'Q3: effect',
                           'question_bbi_2016wave4_basicincome_argumentsfor': 'Q4: aurguments for',
                           'question_bbi_2016wave4_basicincome_argumentsagainst': 'Q5: arguments againts'}, inplace=True)

In [ ]:
# adding quantity column
df_complete['Quantity'] = df_complete.groupby('uuid')['uuid'].transform('count').astype("int64")

In [ ]:
# adding percentage column
# votos por país

list_countries =df_complete['Country'].unique().tolist()
list_countries

In [ ]:
# total people per country column.
df_complete['Total Votes Per Country'] = 0

for country in list_countries:
    df_complete.loc[df_complete['Country'] == country , 'Total Votes Per Country'] = df_complete.loc[df_complete['Country'].str.contains(country) , 'Quantity'].sum()

In [ ]:
df_complete['Percentage'] = (df_complete['Total Votes Per Country'] / df_complete['Total Votes Per Country'].sum()) 
df_complete['Percentage'] = df_complete['Percentage'].astype(float).map(lambda x: '{:.4%}'.format(x)) # adding the %
df_complete

In [ ]:
#Exporting the table for every country
df_result = df_complete[['Country', 'Job Title', 'Age', 'Quantity', 'Percentage']]

In [ ]:
# choosing and printing (should export).

country_choosed = input(print(f'Would you like to see the data for: \n- all countries \n- for: {list_countries}?'))

if country_choosed == 'all countries':
    print('exporting all_countries_data.csv file...')
    df_result.to_csv('../data/result/all_countries_data.csv')
    print('file exported!')
if country_choosed in list_countries:
    df_result_per_country = df_result[df_result['Country'] == (f'{country_choosed}')]
    print('exporting result_per_country.csv file')
    df_result_per_country.to_csv('../data/result/result_per_country.csv')
    print('file exported!')